In [0]:
# 予めやっておくこと
# GPUで動き設定
# メニューから「ランタイム」=>「ランタイムタイプの変更」をクリックして　GPUに設定する

# google　colabからマウントする
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# authorization code取得して入力する（enterを押すとマウント完了）

# 出力メッセージ:
# Enter your authorization code:
# ··········
# Mounted at /content/gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# GitHubからDreamリポジトリをクローンする（既にクローンしているなら実行不要）
!git clone https://github.com/natsutan/dream.git "/content/gdrive/My Drive/Dream"

fatal: destination path '/content/gdrive/My Drive/Dream' already exists and is not an empty directory.


In [0]:
# Git管理されているCSVファイルのディレクトリに移動する
%cd /content/gdrive/My Drive/Dream/labelbox/csv

/content/gdrive/My Drive/Dream/labelbox/csv


In [0]:
# ファイル更新（初回実行の場合は不要）
!git pull origin master

From https://github.com/natsutan/dream
 * branch            master     -> FETCH_HEAD
Updating cba57fa..c8cd974
Fast-forward
 labelbox/csv/export.csv   | 2842 +++++++++++++++++++++++++++++----------------
 labelbox/json/export.json | 2842 +++++++++++++++++++++++++++++----------------
 2 files changed, 3698 insertions(+), 1986 deletions(-)


In [0]:
# CSVファイル展開
import pandas as pd
import re
import json

column_name = ["filename", "lists"]

inputdf = pd.read_csv("export.csv")
index = list(range(0, inputdf.shape[0])) 
outdf = pd.DataFrame(index=index, columns=column_name)

# 写真のファイル名と各パーツのバウbディングボックスの情報をセット
for i in range(inputdf.shape[0]):
  json_data = json.loads(inputdf["Label"][i])
  if not json_data:
    continue

  outdf["filename"][i] = re.search(r'(?<=dog.).*?(?=jpg)', inputdf["External ID"][i]).group(0)
  outdf["lists"][i] = []

  # 各パーツのバウンディングボックス情報をセット
  for j, obj in enumerate(json_data['objects']):
    d = {}
    if ("bbox" in obj):
      d["title"] = obj["value"]
      d.update(**obj["bbox"])
      outdf["lists"][i].append(d)

outdf


,filename,lists
0,65.,"[{'title': 'dog', 'top': 28, 'left': 93, 'heig..."
1,383.,"[{'title': 'left_eye', 'top': 202, 'left': 162..."
2,415.,"[{'title': 'dog', 'top': 40, 'left': 120, 'hei..."
3,479.,"[{'title': 'dog', 'top': 7, 'left': 0, 'height..."
4,95.,"[{'title': 'dog', 'top': 48, 'left': 225, 'hei..."
...,...,...
1844,1848.,"[{'title': 'dog', 'top': 32, 'left': 238, 'hei..."
1845,1880.,"[{'title': 'dog', 'top': 16, 'left': 39, 'heig..."
1846,1944.,"[{'title': 'left_eye', 'top': 92, 'left': 158,..."
1847,1947.,"[{'title': 'dog', 'top': 43, 'left': 43, 'heig..."


In [0]:
'''
事前に犬の写真を保存している共有ディレクトリを自分のドライブにショートカット追加する
ディレクトリを共有していない場合、共有できるようにしてもらう
'''
%cd /content/gdrive/My Drive/パソナ/gog_cat/dog

/content/gdrive/.shortcut-targets-by-id/1aJ9h1h3zJ-qpedHmWDzWbWt_VlTp2utj/gog_cat/dog


In [0]:
import cv2
import glob
import numpy as np
import os
import datetime

# テキストファイル出力用のディレクトリを作成する
dt_now = datetime.datetime.now()
ymdhis = dt_now.strftime('%Y%m%d%H%M%S')
dirname = 'txt_' + ymdhis
if not os.path.isdir(dirname):
  os.mkdir(dirname)

#パスにある犬の写真を全て取得する
jpglist = (glob.glob("dog.*.jpg"))
for i in range(outdf.shape[0]):
  photoname = "dog.{}jpg".format(outdf["filename"][i])
  filename = "{0}/dog.{1}txt".format(dirname, outdf["filename"][i])

  # リストになければ次へ
  if (photoname not in jpglist):
    continue

  try:
    im = cv2.imread(jpglist[i])
  except:
    im = (500, 500)
  im = np.array(im)
  image_width = im.shape[1]  
  image_height = im.shape[0]
  
  content = []
  for j in outdf["lists"][i]:
    # 幅、高さが０.１未満になると警告が出る模様
    x_center = str(int(j["left"])/int(image_width))
    y_center = str(int(j["top"])/int(image_height))
    width = str(int(j["width"])/int(image_width))
    height = str(int(j["height"])/int(image_height))
    if j["title"] == "dog":
      content.append("0 "+x_center+" "+y_center+" "+width+" "+height)

    if j["title"] == "left_eye":
      content.append("1 "+x_center+" "+y_center+" "+width+" "+height)

    if j["title"] == "right_eye":
      content.append("2 "+x_center+" "+y_center+" "+width+" "+height)

  content.sort()

  # テキストファイルに出力
  with open(filename, 'w') as f:
      f.write('\n'.join(content))
